### Stochastic Variational Deep Kernel Learning

https://papers.nips.cc/paper/6426-stochastic-variational-deep-kernel-learning

In [1]:
# code adapted from https://github.com/pytorch/examples/tree/master/mnist

import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import pyro.infer as infer
import pyro.optim as optim
import pyro.poutine as poutine

In [2]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--num-inducing', type=int, default=70, metavar='N',
                    help='number of inducing input (default: 70)')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args("--log-interval 100".split())
args.cuda = not args.no_cuda and torch.cuda.is_available()

pyro.set_rng_seed(args.seed)

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        return x

In [5]:
class Mean(nn.Module):
    def __init__(self):
        super(Mean, self).__init__()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(self.fc1(F.relu(x)))
        x = self.fc2(x)
        return x.t()  # swap latent_shape x num_data

In [6]:
cnn = pyro.module("CNN", CNN().cuda())
mean = pyro.module("Mean", Mean().cuda())

def model(x, y):
    x1 = cnn(x)
    x2 = mean(x1).t()
    x3 = F.softmax(x2, dim=-1)
    pyro.sample("y", dist.Categorical(x3), obs=y)

def guide(x, y):
    pass

optimizer = optim.Adam({"lr": args.lr})

svi = infer.SVI(poutine.scale(model, 1/args.batch_size),
                poutine.scale(guide, 1/args.batch_size),
                optimizer, "ELBO")

def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        pyro.get_param_store().mark_params_active(cnn.parameters())
        pyro.get_param_store().mark_params_active(mean.parameters())
        loss = svi.step(data, target)
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:2.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
            print(pyro.get_param_store().get_param("CNN$$$conv1.bias"))
            print(dict(cnn.named_parameters())['conv1.bias'])
            
for epoch in range(1, 2):
    train(epoch)

Train Epoch:  1 [    0/60000 ( 0%)]	Loss: 2.324166

 0.1863
-0.0779
 0.0171
 0.0792
 0.0140
-0.0902
 0.1054
 0.1329
-0.0216
-0.0346
[torch.cuda.FloatTensor of size (10,) (GPU 0)]


 0.1863
-0.0779
 0.0171
 0.0792
 0.0140
-0.0902
 0.1054
 0.1329
-0.0216
-0.0346
[torch.cuda.FloatTensor of size (10,) (GPU 0)]

Train Epoch:  1 [ 6400/60000 (11%)]	Loss: 0.215937

-0.1187
-0.1382
-0.1197
 0.0319
-0.0400
-0.1270
-0.0518
-0.1471
-0.1413
-0.1569
[torch.cuda.FloatTensor of size (10,) (GPU 0)]


-0.1187
-0.1382
-0.1197
 0.0319
-0.0400
-0.1270
-0.0518
-0.1471
-0.1413
-0.1569
[torch.cuda.FloatTensor of size (10,) (GPU 0)]

Train Epoch:  1 [12800/60000 (21%)]	Loss: 0.169467

-0.3241
-0.2236
-0.2686
-0.1624
-0.0875
-0.2189
-0.2300
-0.2692
-0.2708
-0.2765
[torch.cuda.FloatTensor of size (10,) (GPU 0)]


-0.3241
-0.2236
-0.2686
-0.1624
-0.0875
-0.2189
-0.2300
-0.2692
-0.2708
-0.2765
[torch.cuda.FloatTensor of size (10,) (GPU 0)]

Train Epoch:  1 [19200/60000 (32%)]	Loss: 0.138465

-0.4150
-0.2473
-0.33

In [6]:
cnn = pyro.module("CNN", CNN())
mean = pyro.module("Mean", Mean())
kernel = gp.kernels.RBF(input_dim=320, lengthscale=torch.ones(320))
Xu = cnn(next(iter(train_loader))[0][:args.num_inducing]).detach()
likelihood = gp.likelihoods.MultiClass(num_classes=10)
gpmodel = gp.models.SparseVariationalGP(X=Xu, y=None, kernel=kernel, Xu=Xu,
                                        likelihood=likelihood, mean_function=mean,
                                        latent_shape=torch.Size([10]),
                                        num_data=60000, whiten=True)
if args.cuda:
    cnn.cuda()
    mean.cuda()
    gpmodel.cuda()

optimizer = optim.Adam({"lr": args.lr})

svi = infer.SVI(poutine.scale(gpmodel.model, 1/60000), poutine.scale(gpmodel.guide, 1/60000),
                optimizer, "ELBO")

In [7]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        X = cnn(data)
        gpmodel.set_data(X, target)
        loss = svi.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:2.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [8]:
def test():
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        X = cnn(data)
        f_loc, f_var = gpmodel(X)
        pred = gpmodel.likelihood(f_loc, f_var)
        correct += pred.eq(target).long().cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    with torch.no_grad():
        test()

Train Epoch:  1 [    0/60000 ( 0%)]	Loss: 2.326983
Train Epoch:  1 [ 6400/60000 (11%)]	Loss: 2.329321
Train Epoch:  1 [12800/60000 (21%)]	Loss: 2.331608
Train Epoch:  1 [19200/60000 (32%)]	Loss: 2.363429
Train Epoch:  1 [25600/60000 (43%)]	Loss: 2.314625
Train Epoch:  1 [32000/60000 (53%)]	Loss: 2.351893


Process Process-2:
Traceback (most recent call last):
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/fehiepsi/miniconda3/envs/pyro/lib/python3.5/multiprocessing/connection.

KeyboardInterrupt: 